In [1]:
## Pandas
import pandas as pd
## Numpy
import numpy as np

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

In [3]:
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

In [4]:
rating_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [5]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [6]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [7]:
rating = pd.read_csv(rating_url, sep='\t', low_memory=False)

In [8]:
# Basics
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [9]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9813254 entries, 0 to 9813253
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 673.8+ MB


In [10]:
# Aka's
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [11]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35744241 entries, 0 to 35744240
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.1+ GB


In [12]:
akas = akas.replace({'\\N':np.nan})

In [13]:
# Ratings
rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1966
1,tt0000002,5.8,264
2,tt0000003,6.5,1809
3,tt0000004,5.6,178
4,tt0000005,6.2,2608


In [14]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1307201 entries, 0 to 1307200
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1307201 non-null  object 
 1   averageRating  1307201 non-null  float64
 2   numVotes       1307201 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 29.9+ MB


In [15]:
rating = rating.replace({'\\N':np.nan})

In [16]:
## make columns lowercase for ease of reference
basics.columns = basics.columns.str.lower()

In [17]:
## make columns lowercase for ease of reference
akas.columns = akas.columns.str.lower()

In [18]:
## make columns lowercase for ease of reference
rating.columns = rating.columns.str.lower()

## basics cleaning

In [19]:
# Replacing missing values with nan
basics = basics.replace({'\\N':np.nan})

In [20]:
# Dropping nans from runtime
basics.dropna(subset=['runtimeminutes'], inplace=True)

In [21]:
# Dropping nans from genres
basics.dropna(subset=['genres'], inplace=True)

In [22]:
# Dropping all titletypes not equal to movie
basics.query("titletype == 'movie'", inplace=True)

In [23]:
# Received error on changing startyear to int
# Dropping nan values under startyear in order to change to int
basics.dropna(subset=['startyear'], inplace=True)

In [24]:
# Converting startyear to int in order to drop anything prior to year 2000
basics['startyear'] = basics['startyear'].astype('int')

In [25]:
# Dropping years under 2000
basics = basics[(basics['startyear'] >= 2000) & (basics['startyear'] < 2023)]

In [26]:
basics.head()

,tconst,titletype,primarytitle,originaltitle,isadult,startyear,endyear,runtimeminutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,94,Documentary
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
76059,tt0077684,movie,Histórias de Combóios em Portugal,Histórias de Combóios em Portugal,0,2022,NaN,46,Documentary


In [27]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [28]:
# Filter basics table down/only include the US by using the filter akasdf
keepers =basics['tconst'].isin(akas['titleid'])
keepers

34803      True
61116      True
67669      True
77964      True
86801      True
           ... 
9812927    True
9812936    True
9812975    True
9813020    True
9813104    True
Name: tconst, Length: 147375, dtype: bool

In [29]:
basics = basics[keepers]
basics

,tconst,titletype,primarytitle,originaltitle,isadult,startyear,endyear,runtimeminutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77964,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
9812927,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9812936,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"
9812975,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,NaN,84,Thriller
9813020,tt9916362,movie,Coven,Akelarre,0,2020,NaN,92,"Drama,History"


In [30]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keeper2 =rating['tconst'].isin(akas['titleid'])
keeper2

0           True
1           True
2           True
3           True
4           True
           ...  
1307196     True
1307197     True
1307198    False
1307199    False
1307200    False
Name: tconst, Length: 1307201, dtype: bool

In [31]:
# Dropping all titletypes not equal to movie
akas.query("region == 'US'", inplace=True)

In [32]:
# example making new folder with os
import os
os.makedirs('data/',exist_ok=True) 
# Confirm folder created
os.listdir("data/")

['title.akas.tsv.gz', 'title_basics.csv.gz']

In [33]:
# Calling df 
basics.head(20)

,tconst,titletype,primarytitle,originaltitle,isadult,startyear,endyear,runtimeminutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77964,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama
98043,tt0100275,movie,The Wandering Soap Opera,La Telenovela Errante,0,2017,NaN,80,"Comedy,Drama,Fantasy"
100076,tt0102362,movie,Istota,Istota,0,2000,NaN,80,"Drama,Romance"
101041,tt0103340,movie,Life for Life: Maximilian Kolbe,Zycie za zycie. Maksymilian Kolbe,0,2006,NaN,90,"Biography,Drama"
106105,tt0108549,movie,West from North Goes South,West from North Goes South,0,2004,NaN,96,"Comedy,Mystery"


In [34]:
# Save dataframe to file.
basics.to_csv("data/title_basics.csv.gz",compression='gzip',index=False)

In [35]:
# Open saved file and preview
basics = pd.read_csv("data/title_basics.csv.gz",low_memory = False)
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146725 entries, 0 to 146724
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          146725 non-null  object 
 1   titletype       146725 non-null  object 
 2   primarytitle    146725 non-null  object 
 3   originaltitle   146725 non-null  object 
 4   isadult         146725 non-null  int64  
 5   startyear       146725 non-null  int64  
 6   endyear         0 non-null       float64
 7   runtimeminutes  146725 non-null  int64  
 8   genres          146725 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 10.1+ MB


In [36]:
# Save dataframe to file.
akas.to_csv("data/title.akas.tsv.gz",compression='gzip',index=False)
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1433173 entries, 5 to 35743985
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleid          1433173 non-null  object
 1   ordering         1433173 non-null  int64 
 2   title            1433173 non-null  object
 3   region           1433173 non-null  object
 4   language         3893 non-null     object
 5   types            978227 non-null   object
 6   attributes       46470 non-null    object
 7   isoriginaltitle  1431828 non-null  object
dtypes: int64(1), object(7)
memory usage: 98.4+ MB


In [37]:
# Save dataframe to file.
rating.to_csv("data/title.ratings.tsv.gz",compression='gzip',index=False)
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1307201 entries, 0 to 1307200
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1307201 non-null  object 
 1   averagerating  1307201 non-null  float64
 2   numvotes       1307201 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 29.9+ MB
